In [1]:
file_path = '/tf/haha/all_data/webScrapping-git/' # change this to your file path

Read data from TheGoodScentsCompany
(The data used in paper is obtained on Oct 20th, 2022)

In [2]:
from bs4 import BeautifulSoup
import requests
import pubchempy as pcp
import time
from rdkit import Chem
import re

# Obtain CAS and corresponding webpage

Saving cas and webpage to file 'tgsc_content.txt'

Contents in 'tgsc_content.txt' would be like:

    50-00-0 http://www.thegoodscentscompany.com/data/rw1247381.html
    50-01-1 http://www.thegoodscentscompany.com/data/rw1248841.html
    50-03-3 http://www.thegoodscentscompany.com/data/rw1469201.html
    ...

In [3]:
for page in range(12):
    web_content = 'http://www.thegoodscentscompany.com/allproc-%d.html' % page
    html_content = requests.get(web_content).text
    soup = BeautifulSoup(html_content, 'lxml')

    compounds = soup.find_all('tr')
    compounds = compounds[2:-2]

    cas_list = []
    web_list = []

    for i, compound in enumerate(compounds):
        info = compound.find_all('td')
        
        if len(info) != 2:
            print(i, end=' ')

        cas_list.append(info[0].text)
        
        website = info[1].a['onclick']
        website = website.split('\'')
        web_list.append(website[1])

    f = open(file_path+'tgsc_content.txt', 'a')
    for i in range(len(cas_list)):
        f.write(cas_list[i]+' '+web_list[i]+'\n')
    f.close()

# Open each webpage, find odor descriptors information, save to file

Open each webpage saved in 'tgsc_content.txt'. If there is odor descriptors information on the webpage, save SMILES and odor descriptors to file 'tgsc_smilesOD_1020.txt'

Contents in 'tgsc_smilesOD_1020.txt' would be like:

O=C(O)C(C)O odorless 

C([C@H]([C@H]([C@@H]([C@H](CO)O)O)O)O)O odorless to sweet caramel 

C([C@@H]([C@@H]1C(=C(C(=O)O1)O)O)O)O very mild grassy 

...

In [5]:
# (1020)打开tgsc_content.txt的每个网址， 找od， 保存
web_file = open(file_path+'tgsc_content.txt')
all_line = web_file.readlines()
web_file.close()

no_smi_tgsc = []
for each in range(len(all_line)):

    line = all_line[each]
    line = line.split(' ')
    cas = line[0]
    website = line[1].strip()

    # 查找od
    html_text = requests.get(website).text
    soup = BeautifulSoup(html_text, 'lxml')

    temp_radw5 = soup.find_all('td', class_='radw5')
    od_flag = 0
    radw5_od = []
    for i in range(len(temp_radw5)):
        if 'Odor Description:' in temp_radw5[i]:
            od_flag = 1
            radw5_od.append(i)

    if od_flag == 0:
        continue
    
    ods = ''
    for i in radw5_od:
        print(each, end=' ')
        od_line = temp_radw5[i]
        temp = od_line.find_all('span') #[<span>at 100.00 %. </span>, <span>mild meaty dirty cooked phenolic rubbery<br/>Luebke, William  tgsc, (2006)</span>]
        if len(temp) == 1:
            temp = temp[0]
        else:
            temp = temp[1] # <span>mild meaty dirty cooked phenolic rubbery<br/>Luebke, William  tgsc, (2006)</span>
        temp = temp.stripped_strings
        ods += next(temp)
        if ods[-1] != ' ':
            ods += ' '

    # smiles in tgsc
    try:
        temp = soup.find_all('td', class_='radw11')
        web_smiles = 'http://www.thegoodscentscompany.com/opl/'+temp[0].text+'.html'
        html_text_smi = requests.get(web_smiles).text
        soup_smi = BeautifulSoup(html_text_smi, 'lxml')

        temp_smi = soup_smi.find_all('div', class_='mrado5')
        for i in range(len(temp_smi)):
            if 'SMILES' in temp_smi[i].text:
                break

        smi_line = temp_smi[i]
        smi = smi_line.find('span')
        smiles = smi.text

    except:
        print('%d cannot find smiles in tgsc' % each, end=' ')
        no_smi_tgsc.append(each)
        # cas2smiles
        smiles = pcp.get_properties('CanonicalSMILES', cas, 'name') # [{'CID': 712, 'CanonicalSMILES': 'C=O'}]
        if len(smiles) == 0:
            continue
        smiles = smiles[0]
        smiles = smiles['CanonicalSMILES']

    # 写入文件'tgsc_smilesOD_1020.txt'
    f = open(file_path+'tgsc_smilesOD_1020.txt', 'a')
    f.write(smiles+' '+ods+'\n')
    f.close()

    time.sleep(0.3)

4 13 15 26 32 33 51 58 67 68 69 70 71 76 76 76 76 79 79 79 80 84 88 94 96 96 103 117 119 121 121 121 121 122 122 122 122 122 123 127 130 130 cannot find smiles in tgsc 131 132 136 145 147 153 162 164 165 166 166 167 167 167 167 178 182 182 182 182 182 187 189 192 193 194 196 196 197 199 208 209 217 219 222 223 223 225 226 226 226 231 232 233 238 243 251 251 251 251 251 256 258 258 259 262 262 262 262 262 262 262 267 268 268 275 277 280 281 286 291 294 295 295 295 295 296 306 308 308 308 308 308 309 309 309 309 309 309 313 316 319 319 319 319 319 319 322 323 325 327 328 337 337 337 337 338 338 339 339 339 340 343 343 347 347 347 347 349 349 349 349 349 349 353 354 355 355 355 355 356 358 359 359 360 362 363 363 369 369 369 369 369 372 373 373 375 375 375 375 376 376 380 380 380 387 389 389 389 389 389 390 392 393 393 394 394 394 394 394 394 394 397 398 398 398 398 398 399 404 406 406 406 406 406 408 408 417 417 417 418 418 418 418 419 419 420 420 420 420 421 424 428 430 431 446 446 446 

# Process ODs data

Merge multiple ODs with same meaning to one OD. (For example, merge 'fruity', 'fruit', 'frult-like' to 'fruity')

Delete words that are not ODs (such as 'and', 'very', ...)

Save odorants to file 'tgsc_odorant_1020.txt'

Save odorless molecules to file 'tgsc_odorless_1020.txt'

Contents in 'tgsc_odorant_1020.txt' would be like:

1 OC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO caramel odorless sweet 

2 O=C1O[C@H]([C@@H](O)CO)C(O)=C1O grassy 

3 NCCc1ccc(O)cc1 rubbery cooked meaty phenolic dirty 

...

## similar word (only for ODs that appear more than 30 times)

In [6]:
similar_word = {
            'fruity': ['fruity', 'fruit', 'fruital', 'fruits', 'fruit-like', 'fruitiness', 'frutti', 'fruity-rosy', 'fruit\x92', 'starfruit', 'jackfruit'],
            'green': ['green', 'Green', 'greenery', 'greens', '\x91green'],
            'sweet': ['sweet', 'sweeter', 'sweetness', 'sweetish', 'sweety', 'sweetened', 'sweets'],
            'floral': ['floral', 'florals', 'flower', 'flowery', 'flowers', 'marine-floral', 'citrusyfloral', 'floralcy', 'mayflower', 'floreal'],
            'woody': ['woody', 'wood', 'guaiacwood', 'sandalwood', 'woody-ambery', 'agarwood', 'wormwood', 'woody-lactone', 'rosewood', 'pinewoody', 'andsandalwood', 'woods'],
            'herbaceous': ['herbal', 'herbaceous', 'herb', 'herbs', 'herbacious', 'herbaceousness', 'herbai'],
            'fresh': ['fresh', 'freshly', 'Fresh', 'fresher', 'refreshing', 'freshness', 'refreshingly', 'fresheners', 'freshener'],
            'waxy': ['waxy', 'Waxy', 'wax'],
            'rose': ['rose', 'rosemary', 'rose-like', 'rosey', 'rosy', 'rosewood', 'roses', 'rosewater'],
            'fatty': ['fatty', 'fattiness', 'fat', 'fatly'],
            'spicy': ['spicy', 'spice', 'Spicy', 'allspice'],
            'citrus': ['citrus', 'citric', 'citral', 'citrusy', 'citrusyfloral', 'citralva'],
            'oily': ['oily', 'oil', 'oils', ],
            'tropical': ['tropical', '\x91tropical'],
            'earthy': ['earthy', 'earth', 'Earth'],
            'nutty': ['nutty', 'nut', 'nutmeat', 'peanut', 'walnut', 'nutmeg', 'nuts', 'hazelnuts', 'Peanut', 'peanuts', 'hazelnut', 'walnuts'],
            'apple': ['apple', 'apple-like', 'apples'],
            'balsam': ['balsam', 'balsamic', 'balsmic', 'balm', 'balsamin'],
            'dry': ['dry', 'Dry'],
            'powdery': ['powdery', 'powder', 'powderful'],
            'musty': ['musty', 'Musty'],
            'pineapple': ['pineapple', 'pineapple-like'],
            'roasted': ['roasted', 'roast', 'Roasted', 'roastad'],
            'aldehydic': ['aldehydic', 'aldeydic', 'aldehyde', 'benzaldehyde', 'acetaldehyde', 'aldehydes', 'oxyacetaldehyde', 'aldehyic'],
            'phenolic': ['phenolic', 'acetophenone', 'benzophenone', 'acetophenone-like', 'phenol', 'Phenolic'],
            'creamy': ['creamy', 'creaminess', 'cream'],
            'meaty': ['meat', 'meaty', 'Meaty'],
            'metallic': ['metallic', 'metal', 'metalic'],
            'honey': ['honey', 'honeydew', 'honeysuckle', 'honeylike', 'honney', 'honeyed'],
            'minty': ['minty', 'mint', 'peppermint', 'spearmint', 'mintier', 'cornmint', 'mint-like'],
            'vegetable': ['vegetative', 'vegetable', 'vegetables', 'vegetable-celery', 'vegetal', 'vegetable-like', 'Vegetative'],
            'sulfurous': ['sulfurous', 'sulfuraceous', 'sulphurous', 'Sulfureous', 'sulfury', 'sulfur', 'Sulfury', 'sulfurol', 'sulfureous', 'Sulfuraceous', 'sulfide', 'sulphur'],
            'jasmin': ['jasmin', 'jasmone', 'jasmone-like', 'jasmine', 'isojasmone', '\\u201cjasmin', 'dihydrojasmonate'],
            'banana': ['banana', 'banana-like'],
            'berry': ['juniperberry', 'raspberry', 'berry', 'blueberry', 'strawberry', 'cranberry', 'berry-', 'gooseberry', 'loganberry', 'berry-like', 'bilberry', 'blackberry', 'blueberries', 'raspberries', 'elderberry', 'cloudberry', 'strawberries'],
            'coconut': ['coconut', 'Coconut', 'coco'],
            'pear': ['pear', 'pear-like'],
            'onion': ['onion', 'oniony', 'onion-like', 'onion;', 'onion-garlic', 'onions'],
            'melon': ['melon', 'cucumber-melon', 'melon-like', 'watermelon', 'watermelons', 'melony'],
            'orange': ['orange', 'orangeflower', 'orangeblossom'],
            'amber': ['amber', 'ambergris', 'ambery', 'woody-ambery', 'amberyl'],
            'musk': ['musk', 'musky', 'musks'],
            'animal': ['Animalic', 'animal', 'animalic', 'animal-like;', 'animalistic'],
            'mushroom': ['mushroom', 'mushroom-like', 'mushroomy', 'mushrooms'],
            'natural': ['natural', "'natural'", 'naturally', 'nature'],
            'camphor': ['camphor', 'camphoreous', 'camphorated', 'Camphoraceous', 'camphoraceous', 'camphor-like', 'camphoraceuos', 'camphorerous'],
            'pungent': ['pungent', 'Pungent', 'pungency'],
            'coffee': ['coffee', 'coffee-like'],
            'tobacco': ['tobacco', 'tobacco-leaf', 'tobacco-like'],
            'violet': ['violet-leaf', 'violet', 'orris&violet', 'violetty'],
            'peach': ['peach', 'peach-pit', 'tobacco-like', 'peachy'],
            'caramel': ['caramellic', 'caramel', 'caramelised', '(caramellic', 'caramelized'],
            'leafy': ['leafy', 'leaf', 'violet-leaf', 'cedarleaf', 'tomato-leaf', 'tobacco-leaf', 'leaves'],
            'grape': ['grape', 'grape-like', 'grapes'],
            'cortex': ['cortex'],
            'orris': ['orris', 'orris&violet'],
            'winey': ['wine', 'wine-lee', 'winey', 'winy'],
            'cucumber': ['cucumber', 'cucumber-melon', 'cucumber-like'],
            'plum': ['plum', 'plums'],
            'weedy': ['weedy'],
            'soapy': ['soapy', 'soap', 'soaps'],
            'lemon': ['lemon', 'lemongrass', 'lemon/lime', 'lemony', 'lime'],
            'milky': ['milky', 'milk', 'buttermilk'],
            'alliaceous': ['Alliaceous', 'alliaceous'],
            'leather': ['leathery', 'leather', 'Leathery'],
            'savory': ['savory', 'savoury'],
            'lavender': ['lavender', 'lavandin'],
            'sharp': ['Sharp', 'sharp'],
            'cinnamon': ['cinnamyl', 'cinnamon', 'cinnamic', 'cinnamate', 'cinnamon-wintergreen', 'cinnamomum', 'cinnamony'],
            'cedar': ['cedar', 'cedarwood', 'cedarous', 'cedarleaf'],
            'cognac': ['cognac', 'cognac-like', 'cognac-Iike'],
            'grassy': ['grass', 'grassy', 'lemongrass'],
            'muguet': ['muguet'],
            'grapefruit': ['grapefruit', 'grapfruit'],
            'cheese': ['cheese', 'cheesy', 'cheese-like', 'cheesey', 'Cheese', 'cheeses', 'Cheesy'],
            'pine': ['pine', 'pine-like', 'piney'],
            'hyacinth': ['hyacinth', 'hyacint'],
            'buttery': ['buttery', 'butterscotch', 'butter-like', 'buttermilk', 'butter', 'buttered'],
            'garlic': ['garlic-like', 'garlic', 'Garlic', 'onion-garlic'],
            'vanilla': ['vanilla', 'vanilla-like', 'vanillin', 'vanila'],
            'geranium': ['geranium'],
            'dairy': ['dairy', 'dairy-like'],
            'lily': ['lily', 'lilies'],
            'almond': ['almond', 'almonds', 'almondy'],
            'burnt': ['burnt'],
            'cherry': ['cherry', 'cherry-like', 'cherry-pit'],
            'skin': ['skin', 'skin-like', 'skins'],
            'apricot': ['apricot', 'apricots', 'apricot\x92'],
            'cocoa': ['cocoa', 'roasted,cocoa', 'cocoalike'],
		'medicinal': ['medicinal', 'medical', 'medicated'],
		'celery': ['celery', 'celery-'], 
		'sandalwood': ['sandalwood', 'sandal'],
		'rum': ['rum', 'rummy', 'rumy'],
		'chocolate': ['chocolate', 'chocolate-like'],
		'smoky': ['smoky', 'smoke', 'smoked', 'smokey'],
		'neroli': ['neroli', 'nerol'],
		'ozone': ['ozone', 'ozonic'],
		'potato': ['potato', 'potato-like', 'potatoes'],
		'fishy': ['fishy', 'fish'],
		'lactonic': ['lactonic', 'lactone', 'woody-lactone', 'lactic', 'lactona', 'lactones'],
		'anise': ['anise', 'anisic', 'aniseed', 'anisyl', 'anise-like', 'anis', 'anisaldehyde'],
		'cooling': ['cooling', 'cool', 'coolness'],
		'pepper': ['pepper', 'peppery', 'peppers'], 
		'alcoholic': ['alcoholic', 'alcohol', 'alcohols'],
		'terpene': ['terpene', 'terpenic', 'terpineol', 'terpentine', 'terpenes', 'terpeny', 'terpinolene', 'diterpene', 'terpeney'],
		'aromatic': ['aroma', 'aromatic'],
        'ethereal': ['ethereal', 'etherial', 'ether', 'etheral', 'ethered', 'ethery'],
        'hay': ['hay', 'haylike'],
        'fermented':['fermented', 'fermenty'],
        'juicy': ['juicy', 'juice'], 
        'bergamot': ['bergamot', 'bergamott'],
        'coumarin': ['coumarin', 'coumarinic'],
        'vetiver': ['vetiver', 'vetivert', 'vetiveryl'],
        'clove': ['clove', 'clover', 'cloves'],
        'sweaty': ['sweaty', 'sweat'],
        'peel': ['peel', 'peely', 'peels'], 
        'acidic': ['acidic', 'acid'],
        'bready': ['bready', 'bread', 'gingerbread']
        }

## not odor (for words that appear more than 30 times)

In [7]:
not_odor = ['mild', 'dry', 'bland', 'clean', 'warm', 'sharp', 'powerful', 'ripe', 'strong', 'and', 'very', 'chemical',
 '', 'a', 'with', 'of', 'to', 'like', 'note', 'nuance', 'nuances', 'odor', 'slightly', 'notes', 'diffusive', 
 'the', 'character', 'slight', 'an', 'somewhat', 'in', 'soft', 'undertones', 'brown', 'rich', 'dilution', 'reminiscent',
 'undertone', 'intense', 'has', 'faint', 'characteristic', 'pleasant', 'black', 'is', 'it', 'on', 'red', 'type', 'heavy', 'watery']

## processing and saving to file

In [8]:
# 10月20日! 对tgsc_smilesOD.txt进一步处理并保存
f = open(file_path+'tgsc_smilesOD_1020.txt')
all_line = f.read().split('\n')
f.close()

all_line.pop(-1) #最后一行是\n

smi_od_dict = {}
for line in all_line:
    line1 = line.split(' ')
    smi = line1[0]
    od = line[len(smi):]
    od = re.split('[(|)| |\-|;|,|/|.|\\|:|]',od)

    # smi使用rdkit转为cannical smiles
    try:
        mol = Chem.MolFromSmiles(smi)
        smi = Chem.rdmolfiles.MolToSmiles(mol)
    except:
        smi
        continue

    # 相似词语的处理
    od_new = []
    for i in range(len(od)):
        # od[i] = od[i].strip(',')
        od[i] = od[i].lower()
        if od[i] in not_odor:
            continue
        flag = 0
        for word in similar_word:
            if od[i] in similar_word[word]:
                od_new.append(word)
                flag = 1
        if flag == 0:
            od_new.append(od[i])
    od_new = set(od_new)
    od_new = list(od_new)     

    # 重复smiles合并od
    if smi not in smi_od_dict:
        smi_od_dict[smi] = od_new
    else:
        smi_od_dict[smi].extend(od_new)
        temp = set(smi_od_dict[smi])
        smi_od_dict[smi] = list(temp)

print(len(smi_od_dict))

odorless_smi = []
smi_od_dict1 = smi_od_dict.copy()
for smi in smi_od_dict:
    od = smi_od_dict[smi]
    if len(od) == 1 and od[0] == 'odorless':
        odorless_smi.append(smi)
        smi_od_dict1.pop(smi)

print(len(odorless_smi)) 
print(len(smi_od_dict1)) 

# 将tgsc的smi_od_dict1写入文件
f = open(file_path+'tgsc_odorant_1020.txt', 'w')
count = 1
for smi in smi_od_dict1:
    f.write(str(count)+' ')
    f.write(smi+' ')
    for eachod in smi_od_dict1[smi]:
        f.write(eachod+' ')
    f.write('\n')
    count += 1
f.close()

f = open(file_path+'tgsc_odorless_1020.txt', 'w')
count = 1
for smi in odorless_smi:
    f.write(str(count)+' '+smi+' odorless \n')
    count += 1
f.close()

# 统计tgsc中ods的个数
od_count = {}
for smi in smi_od_dict1:
    od = smi_od_dict1[smi]
    for i in od:
        if i in od_count:
            od_count[i] += 1
        else:
            od_count[i] = 1
od_count_sort = sorted(
            od_count.items(), key=lambda x: x[1], reverse=True)
print(od_count_sort)

RDKit ERROR: [08:15:40] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
RDKit ERROR: [08:15:40] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'
RDKit ERROR: [08:15:40] SMILES Parse Error: syntax error while parsing: SJWFXCIHNDVPSH-MRVPVSSYSA-N
RDKit ERROR: [08:15:40] SMILES Parse Error: Failed parsing SMILES 'SJWFXCIHNDVPSH-MRVPVSSYSA-N' for input: 'SJWFXCIHNDVPSH-MRVPVSSYSA-N'
RDKit ERROR: [08:15:40] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
RDKit ERROR: 
RDKit ERROR: [08:15:40] Explicit valence for atom # 1 B, 5, is greater than permitted
RDKit ERROR: [08:15:40] SMILES Parse Error: syntax error while parsing: H2S
RDKit ERROR: [08:15:40] SMILES Parse Error: Failed parsing SMILES 'H2S' for input: 'H2S'
RDKit ERROR: [08:15:40] Explicit valence for atom # 1 B, 5, is greater than permitted
RDKit ERROR: [08:15:41] Explicit valence for atom 

4465
222
4243
[('fruity', 1334), ('sweet', 1186), ('green', 1118), ('floral', 907), ('woody', 695), ('herbaceous', 585), ('fresh', 576), ('fatty', 432), ('spicy', 411), ('waxy', 404), ('citrus', 341), ('rose', 335), ('nutty', 292), ('oily', 289), ('earthy', 275), ('balsam', 268), ('berry', 245), ('tropical', 241), ('sulfurous', 216), ('aromatic', 212), ('vegetable', 210), ('apple', 200), ('minty', 199), ('powdery', 196), ('roasted', 183), ('meaty', 176), ('pineapple', 176), ('creamy', 167), ('aldehydic', 167), ('ethereal', 162), ('musty', 160), ('phenolic', 157), ('leafy', 153), ('honey', 144), ('caramel', 140), ('camphor', 140), ('pungent', 133), ('metallic', 130), ('orange', 127), ('winey', 126), ('pear', 124), ('melon', 120), ('natural', 118), ('banana', 114), ('amber', 113), ('buttery', 107), ('jasmin', 107), ('onion', 105), ('cheese', 103), ('musk', 102), ('animal', 98), ('coffee', 97), ('lemon', 96), ('coconut', 95), ('pine', 90), ('burnt', 89), ('vanilla', 88), ('peach', 88), ('

RDKit ERROR: [08:15:41] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
RDKit ERROR: [08:15:41] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'
